In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip /content/drive/MyDrive/Kaggle/140k-real-and-fake-faces.zip

In [ ]:
import tensorflow.keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Lambda, concatenate
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Conv2D,Conv3D, BatchNormalization, Activation, MaxPooling2D, MaxPooling3D, GlobalAveragePooling2D, 
                          Dense, Flatten, Dropout)
from tensorflow.keras.layers import  Dropout , BatchNormalization , Dense
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from tensorflow.keras.callbacks import Callback , ReduceLROnPlateau , ModelCheckpoint, CSVLogger
from sklearn.metrics import cohen_kappa_score, accuracy_score , f1_score, recall_score, precision_score
from tensorflow.keras.losses import categorical_crossentropy as logloss
from tensorflow.keras.metrics import categorical_accuracy
import numpy as np 
import pandas as pd 
from sklearn.metrics import classification_report , confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline
import os
import cv2
from PIL import Image
import scipy
import tensorflow as tf
from sklearn import metrics
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   rotation_range=360,
                                   shear_range=0.2,
                                   zoom_range=0.2
                                  )
training_set= train_datagen.flow_from_directory('/content/real_vs_fake/real-vs-fake/train',
                                                 class_mode='binary',
                                                 shuffle=True,
                                                 target_size=(256,256),
                                                 batch_size=64
                                                )

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255
                                  )
testing_set = test_datagen.flow_from_directory('/content/real_vs_fake/real-vs-fake/test',
                                                 class_mode='binary',
                                                 shuffle=True,
                                                 target_size=(256,256),
                                                 batch_size=64
                                                )

In [ ]:
valid_datagen = ImageDataGenerator(rescale=1./255
                                  )
valid_set = valid_datagen.flow_from_directory('/content/real_vs_fake/real-vs-fake/valid',
                                                 class_mode='binary',
                                                 shuffle=True,
                                                 target_size=(256,256),
                                                 batch_size=64
                                                )

In [ ]:
training_set.class_indices

In [ ]:
CNN = tf.keras.models.Sequential()
CNN.add(tf.keras.layers.Conv2D(filters=8, kernel_size=3, activation='LeakyReLU', input_shape=[256,256,3]))
CNN.add(BatchNormalization())

CNN.add(tf.keras.layers.Conv2D(filters=16, kernel_size=3, activation='LeakyReLU'))
CNN.add(tf.keras.layers.Conv2D(filters=16, kernel_size=3, activation='LeakyReLU'))
CNN.add(BatchNormalization())
CNN.add(tf.keras.layers.AveragePooling2D(pool_size=2, strides=2))

CNN.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='LeakyReLU'))
CNN.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='LeakyReLU'))
CNN.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='LeakyReLU'))
CNN.add(BatchNormalization())
CNN.add(tf.keras.layers.AveragePooling2D(pool_size=2, strides=2))

CNN.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='LeakyReLU'))
CNN.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='LeakyReLU'))
CNN.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='LeakyReLU'))
CNN.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='LeakyReLU'))
CNN.add(BatchNormalization())
CNN.add(tf.keras.layers.AveragePooling2D(pool_size=2, strides=2))

#Max
CNN.add(tf.keras.layers.Conv2D(filters=128, kernel_size=5, activation='LeakyReLU'))
CNN.add(BatchNormalization())
CNN.add(tf.keras.layers.MaxPooling2D(pool_size=2, strides=2))
#Max
CNN.add(tf.keras.layers.Conv2D(filters=256, kernel_size=5, activation='LeakyReLU'))
CNN.add(BatchNormalization())
CNN.add(tf.keras.layers.MaxPooling2D(pool_size=2, strides=2))

CNN.add(tf.keras.layers.Flatten())
CNN.add(Dropout(0.5))


CNN.add(tf.keras.layers.Dense(units=32, activation='LeakyReLU'))
CNN.add(Dropout(0.5))

CNN.add(tf.keras.layers.Dense(units=16, activation='LeakyReLU'))
CNN.add(Dropout(0.5))

CNN.add(tf.keras.layers.Dense(units=16, activation='LeakyReLU'))
CNN.add(Dropout(0.5))
CNN.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))


In [ ]:
CNN.compile(optimizer='adam', metrics=['accuracy'], loss='binary_crossentropy')

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Define the paths for saving the best model weights and early stopping patience
checkpoint_path = "/content/drive/MyDrive/CNN Epoch.h5"
early_stopping_patience = 3

# Define the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    checkpoint_path,
    monitor='val_loss',
    save_best_only=False,
    mode="min",
    verbose=3
)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', 
                              factor=0.2, 
                              patience=3, 
                              verbose=3, 
                              min_delta=0.0001
                             )
# Define the EarlyStopping callback
early_stopping_callback = EarlyStopping(
    patience=early_stopping_patience,
    monitor="val_loss",
    mode="min",
    verbose=3
)

In [ ]:
history =CNN.fit(training_set, batch_size=64, epochs=10, validation_data=(valid_set), callbacks=[early_stopping_callback ,checkpoint_callback,reduce_lr ])

In [ ]:
fig = plt.figure()
plt.plot(history.history['accuracy'], color='teal', label='accuracy')
plt.plot(history.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="lower right")
plt.show()


In [ ]:
fig = plt.figure()
plt.plot(history.history['loss'], color='teal', label='loss')
plt.plot(history.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
# Evaluate the model on the test data
loss, accuracy = CNN.evaluate(testing_set)
# Print the loss and accuracy
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")

In [ ]:
#Proposed CNN model 
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(filters=8, kernel_size=3, activation='ReLU', input_shape=[256,256,3]))
model.add(BatchNormalization())

model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='ReLU', padding="same"))
model.add(BatchNormalization())
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='ReLU', padding="same"))
model.add(BatchNormalization())
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='ReLU', padding="same"))
model.add(BatchNormalization())
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='ReLU', padding="same"))
model.add(BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=2, strides=2))
model.add(Dropout(0.25))

model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='ReLU', padding="same"))
model.add(BatchNormalization())
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='ReLU', padding="same"))
model.add(BatchNormalization())
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='ReLU', padding="same"))
model.add(BatchNormalization())
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='ReLU', padding="same"))
model.add(BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=2, strides=2))
model.add(Dropout(0.25))

model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='ReLU', padding="same"))
model.add(BatchNormalization())
model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='ReLU', padding="same"))
model.add(BatchNormalization())
model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='ReLU', padding="same"))
model.add(BatchNormalization())
model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='ReLU', padding="same"))
model.add(BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=2, strides=2))
model.add(Dropout(0.5))



#Flattend
model.add(tf.keras.layers.Flatten())

model.add(layers.Dense(128, activation="relu"))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(64, activation="relu"))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))





In [ ]:
model.compile(optimizer='adam', metrics=['accuracy'], loss='binary_crossentropy')

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Define the paths for saving the best model weights and early stopping patience
checkpoint_path = "/content/drive/MyDrive/NEW MODEL EPOCH.h5"


# Define the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    checkpoint_path,
    monitor='val_loss',
    save_best_only=False,
    mode="min",
    verbose=3
)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', 
                              factor=0.2, 
                              patience=3, 
                              verbose=3, 
                              min_delta=0.0001
                             )
# Define the EarlyStopping callback
early_stopping_callback = EarlyStopping(
    patience=3,
    monitor="val_loss",
    mode="min",
    verbose=3
)

In [ ]:
history1 =model.fit(training_set, batch_size=64, epochs=10, validation_data=(valid_set), callbacks=[early_stopping_callback ,checkpoint_callback,reduce_lr ])

In [ ]:
fig = plt.figure()
plt.plot(history1.history['accuracy'], color='teal', label='accuracy')
plt.plot(history1.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="lower right")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(history1.history['loss'], color='teal', label='loss')
plt.plot(history1.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
# Evaluate the model on the test data
loss, accuracy = CNN.evaluate(testing_set)
# Print the loss and accuracy
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")